<a href="https://colab.research.google.com/github/fdmy2713-dotcom/FDS/blob/main/1_LinearRegression_Marketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import data

In [4]:
##import os
##os.chdir('C:\\Users\\taiho\\Documents\\D_Drive\\Teaching Assignments\\PSB_Academy\\Hertfordshire_Course\\2_BSc_Intro_ML_and_NN\\4_Personal_Resources\\2_Codes\\data')

import pandas as pd
##df = pd.read_csv('marketing.csv')
##df.head()

In [5]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/MSc in Data Science/FDS/Shared/02_Codes/data/marketing.csv'
df = pd.read_csv(file_path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,youtube,facebook,newspaper,sales
0,276.12,45.36,83.04,26.52
1,53.40,47.16,54.12,12.48
2,20.64,55.08,83.16,11.16
3,181.80,49.56,70.20,22.20
4,216.96,12.96,70.08,15.48


## 2. Train-Test split (without sklearn)

In [ ]:
## Randomizes the data
df = df.sample(frac=1, random_state=42)
df.head()

,youtube,facebook,newspaper,sales
95,195.96,37.92,63.48,20.28
15,234.48,57.24,63.48,26.88
30,351.48,33.96,51.84,25.68
158,14.04,44.28,54.24,8.76
128,264.36,58.80,3.84,29.64


In [ ]:
## select split ratio
ratio = 0.8

## train_size
train_size = int(ratio*len(df))

## split data into training and testing set
df_train = df[0:train_size]
df_test = df[train_size:].copy()  ## independent dataframe (instead of a view)

## check on training and test size
print(f'training set size: {len(df_train)}')
print(f'testing set size: {len(df_test)}')

training set size: 160
testing set size: 40


## 3.  Modelling & Evaluation of model performance

### (a) Method 1: using sklearn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## initiate linear regression model
model = LinearRegression()

## define predictor and response variables
X, y = df_train[['youtube', 'facebook', 'newspaper']], df_train['sales']

## fit regression model, fit=train
model.fit(X,y)

## display regression coefficients and R-squared value of the moodel
print(model.intercept_, model.coef_, model.score(X,y))

3.7221271206471993 [ 0.04556078  0.19072034 -0.00660746] 0.9061768033395261


In [ ]:
### Refining the model
X, y = df_train[['youtube', 'facebook']], df_train['sales']

model.fit(X,y)

print(model.intercept_, model.coef_, model.score(X,y))

3.6004265392842996 [0.04545172 0.18726581] 0.9054684776114337


In [ ]:
## forecasting using trained model
predicted_values = model.predict(df_test[['youtube', 'facebook']])
df_test.loc[:, 'predicted'] = predicted_values

# summarize the fit of the model
linear_mape = round(mean_absolute_percentage_error(df_test['sales'], df_test['predicted']),2)
linear_mse = round(mean_squared_error(df_test['sales'], df_test['predicted']),2)
linear_r2 = round(r2_score(df_test['sales'], df_test['predicted']),2)

print(f'The model Mean Absolute Percentage Error: {linear_mape}')
print(f'The model Mean Squared Error: {linear_mse}')
print(f'The model r-squared: {linear_r2}')

The model Mean Absolute Percentage Error: 0.22
The model Mean Squared Error: 5.43
The model r-squared: 0.87


### (b) ASIDE:  Method 2: Using statsmodels (an alternative to sklearn)

In [ ]:
import statsmodels.api as sm

## define response variable
y = df_train['sales']

## define predictor variables
x = df_train[['youtube', 'facebook', 'newspaper']]

## add a constant to predictor variables
x = sm.add_constant(x)

## fit linear regression model
model = sm.OLS(y, x).fit()

## view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.904
Method:                 Least Squares   F-statistic:                     502.2
Date:                Tue, 11 Feb 2025   Prob (F-statistic):           6.60e-80
Time:                        20:46:05   Log-Likelihood:                -330.17
No. Observations:                 160   AIC:                             668.3
Df Residuals:                     156   BIC:                             680.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.7221      0.392      9.504      0.0

## 4. Making sense of the predicted results

In [ ]:
df_test.head()

,youtube,facebook,newspaper,sales,predicted
175,332.28,58.68,50.16,32.40,29.691881
134,44.28,46.32,78.72,12.96,14.287181
167,248.16,6.24,23.28,14.64,16.048264
183,345.12,51.60,86.16,31.44,28.949640
63,123.24,35.52,10.08,16.80,15.853578


In [ ]:
print(model.intercept_, model.coef_, model.score(X,y))

3.6004265392842996 [0.04545172 0.18726581] 0.9054684776114337


In [ ]:
## for first predicted result in the testing data:
predicted_1 = 3.6004265392842996 + (0.04545172*332.28) + (0.18726581*58.68)
print(predicted_1)

29.6918817916843


prediction = 3.6 + 0.046*(youtube) + 0.187*(facebook)

## 5. Model Evaluation

### Mean Squared Error (MSE)

In [ ]:
df_test['sq_error'] = (df_test['sales'] - df_test['predicted']) ** 2
df_test.head()

,youtube,facebook,newspaper,sales,predicted,sq_error
175,332.28,58.68,50.16,32.40,29.691881,7.333906
134,44.28,46.32,78.72,12.96,14.287181,1.761409
167,248.16,6.24,23.28,14.64,16.048264,1.983207
183,345.12,51.60,86.16,31.44,28.949640,6.201895
63,123.24,35.52,10.08,16.80,15.853578,0.895715


In [ ]:
print(f'Our calculated mean squared error (mse): {df_test["sq_error"].mean()}')
print(f'Earlier calculated mse by sklearn: {linear_mse}')

Our calculated mean squared error (mse): 5.427346751621543
Earlier calculated mse by sklearn: 5.43


### Mean Absolute Percentage Error (MAPE)

In [ ]:
df_test['APE'] = abs((df_test['sales'] - df_test['predicted']) / df_test['sales']) * 100
df_test.head()

,youtube,facebook,newspaper,sales,predicted,sq_error,APE
175,332.28,58.68,50.16,32.40,29.691881,7.333906,8.358391
134,44.28,46.32,78.72,12.96,14.287181,1.761409,10.240594
167,248.16,6.24,23.28,14.64,16.048264,1.983207,9.619288
183,345.12,51.60,86.16,31.44,28.949640,6.201895,7.920994
63,123.24,35.52,10.08,16.80,15.853578,0.895715,5.633465


In [ ]:
print(f'Our calculated mean abolsute percentage error (MAPE): {df_test["APE"].mean()}')
print(f'Earlier calculated MAPE by sklearn: {linear_mape}')

Our calculated mean abolsute percentage error (MAPE): 22.24182566229485
Earlier calculated MAPE by sklearn: 0.22


### R-Squared (R^2)

In [ ]:
# Calculate the mean of 'sales'
mean_sales = df_test['sales'].mean()

# Calculate SST (Total Sum of Squares)
df_test['SST'] = (df_test['sales'] - mean_sales) ** 2

# Calculate SSR (Sum of Squares of Residuals)
df_test['SSR'] = (df_test['sales'] - df_test['predicted']) ** 2

# Calculate R-squared
r2 = 1 - df_test['SSR'].sum() / df_test['SST'].sum()

df_test.head()

,youtube,facebook,newspaper,sales,predicted,sq_error,APE,SST,SSR
175,332.28,58.68,50.16,32.40,29.691881,7.333906,8.358391,238.795209,7.333906
134,44.28,46.32,78.72,12.96,14.287181,1.761409,10.240594,15.896169,1.761409
167,248.16,6.24,23.28,14.64,16.048264,1.983207,9.619288,5.322249,1.983207
183,345.12,51.60,86.16,31.44,28.949640,6.201895,7.920994,210.047049,6.201895
63,123.24,35.52,10.08,16.80,15.853578,0.895715,5.633465,0.021609,0.895715


In [ ]:
print(f'Our calculated R-Squared: {r2}')
print(f'Earlier calculated R-Squared by sklearn: {linear_r2}')

Our calculated R-Squared: 0.8651221957423947
Earlier calculated R-Squared by sklearn: 0.87
